# Steps for Installation

(1) Create a local directory

(2) Open a terminal at the directory location

(3) Install poetry: `pip install poetry`

(4) Within the directory where you installed poetry, create a `pyproject.toml` file. It MUST be called `pyproject.toml`. Populate the file with the following:

```
[tool.poetry]
name = "comp-4800-testing"
version = "0.1.0"
description = ""
authors = [""]
readme = "README.md"

[tool.poetry.dependencies]
python = ">=3.11,<=3.13"
crewai = "^0.30.5"
langchain = "^0.1.20"
ollama = "^0.2.0"
jupyter = "^1.0.0"
ipykernel = "^6.29.4"
llama-cpp-python = "^0.2.0"  


[build-system]
requires = ["poetry-core"]
build-backend = "poetry.core.masonry.api"
```

(5) In terminal, run `poetry install`. This command installs all dependencies listed in `pyproject.toml` within a virtual environment, ensuring all required packages are available and do not conflict with other projects.

(6) Create a Jupyter kernel linked to this new environment by running the following: 

```
poetry run python -m ipykernel install --user --name=comp_4800_llm_testing --display-name="Python (COMP 4800)"
```

(7) Start Jupyter Notebook from within this virtual environment: `poetry run jupyter notebook`

In [3]:
import time

from textwrap import dedent
from langchain_community.llms import Ollama
from crewai import Crew, Agent, Task

print("All imports successful")

All imports successful


In [4]:
# Initialize Llama3 LLM for agents
'''
This accesses the local model that I created via a Modelfile.
When using the `invoke()` method, it correctly responds with
the SYSTEM prompt I've set.

However, it seems this information is lost when using a crew.
'''
llm = Ollama(model="NEW_MODEL")
# ollama3.invoke("Tell me a joke")

print(f"{llm.model} initialized successfully")

NEW_MODEL initialized successfully


# Create Agents

In [5]:
class FunctionTestAgents:

    def variable_name_agent(self):
        return Agent(
            role='Variable Name Agent',
            goal='Ensure that variable names are descriptive, concise, and accurate.',
            backstory=dedent(
                '''
                You're an expect agent that helps developers write clean, efficient code.
                Your job is to ensure that variable names are descriptive, concise, and accurate.
                Do not add comments to the code.
                '''),
            allow_delegation=False,
            verbose=True,
            llm=llm
    )

    def code_comment_agent(self):
        return Agent(
            role='Code Comment Agent',
            goal='Add comments to code to explain its purpose and functionality. Only add comments that are necessary for a user to understand the code.',
            backstory=dedent(
                '''
                You're an expect code commentor. Your job is to add comments to the code to explain its purpose and functionality. Only add comments that
                are necessary for a user to understand the code. You're aware that code should be self-explanatory whenever possible, and that comments
                should only be used to explain complex or non-obvious parts of the code.
                '''),
            allow_delegation=False,
            verbose=True,
            llm=llm
    )

    def documentation_agent(self):
        return Agent(
            role='Documentation Agent',
            goal='Document code in a way that is easy to understand and follow.',
            backstory=dedent(
                '''
                You're an expert in writing docstrings. Your documentation is clear, concise, accurate, and easy to understand.
                '''),
            allow_delegation=False,
            verbose=True,
            llm=llm
    )

# Create Tasks

In [6]:
class FunctionTestTasks:

    def improve_variable_names(self, agent, request):
        return Task(
            description=dedent(
                f'''
                Receive Python code. Ensure that all variable names are descriptive, concise, and accurate.
                Do not add comments to the code.

                Code
                ----
                {request}
                '''),
            expected_output=dedent(
                '''
                Python code with improved variable names
                '''),
                agent=agent
        )

    def add_code_comments(self, agent, request):
        return Task(
            description=dedent(
                f'''
                Receive Python code. Add comments to the code to explain its purpose and functionality.
                Only add comments that are necessary for a user to understand the code. If a line of
                code is self-explanatory, do not add a comment.

                Code
                ----
                {request}
                '''),
            expected_output=dedent(
                '''
                Python code with added comments
                '''),
                agent=agent
        )

    def review_code_comments(self, agent, request):
        return Task(
            description=dedent(
                f'''
                Receive Python code with comments. Review the code and ensure that all comments are
                accurate and helpful. If a comment is not necessary, remove it.

                Code
                ----
                {request}
                '''),
            expected_output=dedent(
                '''
                Python code with reviewed comments
                '''),
                agent=agent
        )

    def add_documentation(self, agent, request):
        return Task(
            description=dedent(
                f'''
                Add docstrings to the code to explain its purpose and functionality.
                Ensure the documentation is clear, concise, accurate, and easy to understand.

                Code
                ----
                {request}
                '''),
            expected_output=dedent(
                '''
                Python code with added documentation
                '''),
                agent=agent
        )

# Main Code

In [7]:
# Initialize Agents, Tasks
agents = FunctionTestAgents()
tasks = FunctionTestTasks()

# Create agents
variable_name_agent = agents.variable_name_agent()
# code_comment_agent = agents.code_comment_agent()
# documentation_agent = agents.documentation_agent()

print("##################################")
print("## Welcome to the Function Crew ##")
print("##################################")

game_request = input("Enter the code you want to improve: ")
start_time = time.perf_counter()

# Create task for breaking project down into tasks
improve_variable_names_task = tasks.improve_variable_names(variable_name_agent, game_request)
# add_code_comments_task = tasks.add_code_comments(variable_name_agent, improve_variable_names_task)
# add_documentation_task = tasks.add_documentation(variable_name_agent, improve_variable_names_task)

# Create crew and assign tasks
crew = Crew(
    agents = [
        variable_name_agent,
#         documentation_agent
    ],
    tasks = [
        improve_variable_names_task,
#         add_documentation_task
    ],
    verbose=True
)

output = crew.kickoff()
end_time = time.perf_counter()

print("################################")
print("## Refactored Function Output ##")
print("################################")
print(f"## Time taken: {end_time - start_time:.2f} seconds\n")
print(output)


##################################
## Welcome to the Function Crew ##
##################################
Enter the code you want to improve: def print:
 [DEBUG]: == Working Agent: Variable Name Agent
 [INFO]: == Starting Task: 
Receive Python code. Ensure that all variable names are descriptive, concise, and accurate.
Do not add comments to the code.

Code
----
def print:



> Entering new CrewAgentExecutor chain...
Thought: I'm ready to help improve the variable names in this Python code.

Final Answer:
```
def print_message(message_to_print):
    printed_lines = []
    for line in message_to_print.splitlines():
        printed_lines.append(line.strip())
    formatted_message = '\n'.join(printed_lines)
    return formatted_message
```

> Finished chain.
 [DEBUG]: == [Variable Name Agent] Task output: ```
def print_message(message_to_print):
    printed_lines = []
    for line in message_to_print.splitlines():
        printed_lines.append(line.strip())
    formatted_message = '\n'.join